In [45]:
import urllib.request
from bs4 import BeautifulSoup
import os
import re

# Функция преобразовывает буквенный месяц
# (дата на сайте указана в формате ДД МЕСЯЦ (ГГГГ)) в числовой

def monthToDigit(month):
    if month == 'января':
        monthDigit = '01'

    elif month == 'февраля':
        monthDigit = '02'

    elif month == 'марта':
        monthDigit = '03'

    elif month == 'апреля':
        monthDigit = '04'

    elif month == 'мая':
        monthDigit = '05'

    elif month == 'июня':
        monthDigit = '06'

    elif month == 'июля':
        monthDigit = '07'

    elif month == 'августа':
        monthDigit = '08'

    elif month == 'сентября':
        monthDigit = '09'

    elif month == 'октября':
        monthDigit = '10'

    elif month == 'ноября':
        monthDigit = '11'

    elif month == 'декабря':
        monthDigit = '12'

    return monthDigit

# Функция поиска (мета)данных по тэгам


def meta(pageUrl):
    try:
        d = {}
        page = urllib.request.urlopen(pageUrl)
        text = page.read().decode('utf-8')
        soup = BeautifulSoup(text, 'html.parser')
        # поиск заголовка
        d['header'] = soup.find('div', {'class': 'f26'}).get_text()
        # поиск текста статьи
        tagText = soup.find('div', {'id': 'material_text'}).get_text()
        # чистим статью от тэгов html, которые попали в div
        woN = re.sub(r'\n', '', tagText)
        woR = re.sub(r'\r', '', woN)
        woT = re.sub(r'\t', '', woR)
        # чистый текст
        d['text'] = woT.strip()
        # здесь содержится информация об авторе и дате
        footer = soup.find('div', {'id': 'vnizu_a_n_v'})
        children = footer.findChildren('div')
        # строка с именем автора
        authorLine = str(children[0])
        # строка с датой
        dateLine = str(children[1])
        # поиск автора
        author = re.search(
            'Автор: (.*?)(((\.)|(\,))(.*?))?</div>', authorLine)
        linkAuthor = re.search(
            '<a.+?></a>', authorLine)   
        if author and not linkAuthor:
            d['author'] = author.group(1)
        else:
            d['author'] = 'None'
        # поиск даты
        date = re.search('Время: (.*?)</div>', dateLine)
        if date:
            # выделяем год
            year = re.search('[а-я]*? (....) г', date.group(1))
            if year:
                d['year'] = year.group(1)
            # но иногда его нет...
            else:
                d['year'] = 'None'
            # выделяем месяц
            month = re.search('.. ([а-я]*)[ ]*?(.... г)?', date.group(1))
            if month:
                digitMonth = month.group(1)
                d['month'] = monthToDigit(digitMonth.strip())
            # выделяем день
            day = re.search('(..) [а-я]*[ ]*?(.... г)?', date.group(1))
            if day:
                d['day'] = day.group(1)

            return d
    # на случай, если страница сломана
    except:
        print('Error at', pageUrl)

        
        # собственно краулер


def crowler():
    # ссылка на странички
    commonUrl = 'http://www.vecherniy.com/wall?id='
    # огромный словарь всех статей с сайта
    # ключ - номер статьи на сайте,
    # значения - словарь с мета-данными каждой статьи
    bigD = {}
    # считаем статьи
#     изменила количество страниц
    for i in range(685, 690):
        pageUrl = commonUrl + str(i)
        d = meta(pageUrl)
        if d:
            # если все прошло успешно, и ссылка не сломана -
            # добавляем в мета-данные адрес
            d['url'] = pageUrl
            bigD[i] = d
        # если нет - ничего не делаем
        else:
            pass
    return bigD

a = crowler()

In [46]:
def mystemization(root, bigD, abbr, special):
    # местоположение mystem
#     изменила путь к mystem
    way = r'C:\Users\annap\OneDrive\Рабочий стол\mystem.exe'
    # функции mystem
    settings = ' -cdli --eng-gr '
    for key, values in bigD.items():
        mystemPath = root
        # создаем папку с годом, месяцем, туда записываем статьи
        dirPYMPath = os.path.join(root, bigD[key]['year'], bigD[key]['month'])
        # для первой папки год\месяц
        if not os.path.exists(dirPYMPath):
            os.makedirs(dirPYMPath)
        # каждое предложение с новой строчки
        text = break_maker(bigD[key]['text'])
        # временный файл txt, откуда мы будем брать текст для разметки
        filename = 'input' + str(bigD[key]['ordinal']) + '.txt'
        filepath = os.path.join(dirPYMPath, filename)
        # создаем новый файл и записываем туда текст статьи
        with open(filepath, 'x', encoding='utf-8') as f:
            f.write(text)
        # майстемизация в деле:
        # временный файл
        inp = filepath
        # перманентный файл: articleN.xml
        outfilename = 'article' + str(bigD[key]['ordinal']) + abbr
        # путь к файлу в папке mystem
        out = os.path.join(dirPYMPath, outfilename)
        os.system(way + settings + special + inp + ' ' + out)
        # удаляем временный файл
        os.remove(filepath)

In [ ]:
def mystemization():
    # местоположение mystem
#     изменила путь к mystem
    way = r'C:\Users\annap\OneDrive\Рабочий стол\mystem.exe'
    # функции mystem
    settings = ' -cdli --eng-gr '
    for key, values in bigD.items():
        mystemPath = root
        # создаем папку с годом, месяцем, туда записываем статьи
        dirPYMPath = os.path.join(root, bigD[key]['year'], bigD[key]['month'])
        # для первой папки год\месяц
        if not os.path.exists(dirPYMPath):
            os.makedirs(dirPYMPath)
        # каждое предложение с новой строчки
        text = break_maker(bigD[key]['text'])
        # временный файл txt, откуда мы будем брать текст для разметки
        filename = 'input' + '1' + '.txt'
        filepath = os.path.join(dirPYMPath, filename)
        # создаем новый файл и записываем туда текст статьи
        with open(filepath, 'x', encoding='utf-8') as f:
            f.write(text)
        # майстемизация в деле:
        # временный файл
        inp = filepath
        # перманентный файл: articleN.xml
        outfilename = 'article' + str(bigD[key]['ordinal']) + abbr
        # путь к файлу в папке mystem
        out = os.path.join(dirPYMPath, outfilename)
        os.system(way + settings + special + inp + ' ' + out)
        # удаляем временный файл
        os.remove(filepath)

In [51]:
def mystemization():
    way = r'C:\Users\annap\OneDrive\Рабочий стол\mystem.exe'
    settings = ' -cdli --eng-gr '
    inp = r'C:\Users\annap\OneDrive\Рабочий стол\prog18\homeworks\hw3\YV\plain\2015\03\article2.txt'
    out = r'C:\Users\annap\OneDrive\Рабочий стол\prog18\homeworks\hw3\YV\plain\2015\03\article3.txt'
    os.system(way + settings + '.txt' + inp + ' ' + out)
    

mystemization()
